# Variational Quantum Eigensolver

$$𝐻=X𝑋+Z𝑍+𝑋𝑌. $$

Noticing that:
$$
X=HZH\quad Y=(HS^\dagger)^\dagger Z(HS^\dagger)
$$

## VQE

We now proceed implementing the VQE architecure, which works as follows:
* choose an ansatz for a trial state $|\psi(\theta)\rangle$, parametrized by the parameter $\theta$
* use three different quantum circuits to estimate the mean values $\langle X_1 X_2 \rangle_\theta$, $\langle Y_1 Y_2 \rangle_\theta$ and $\langle Z_1 Z_2 \rangle_\theta$, where the subscript indicate the dependece on $\theta$. 
* compute the energy $E(\theta)=\langle \mathcal{H} \rangle_\theta$
* change $\theta$ in order to reach a lower energy

As suggested, we use the ansatz $|\psi(\theta)\rangle = [(RX(\theta)_1\otimes \mathbb{1}_2) \ \text{CNOT}\ (H_1\otimes \mathbb{1}_2)]|00\rangle$.

The quantum circuits are built with **Qiskit**.

###### Importation of packages

In [2]:
import qiskit as qk
from qiskit import Aer
import numpy as np
from scipy.optimize import minimize_scalar, minimize, optimize
from numpy import pi

sim_bknd = qk.Aer.get_backend('qasm_simulator')

###### Functions declaration

In [36]:
def ansatz(qc, qr, params):
    """
    Builds the trial state using the ansatz: (I Sdg) (I RZ ) (RY I) CX (H I)|00>
    
    Arguments
    -----------
    qc: is a QuantumCircuit object from Qiskit
    qr: is a QuantumRegister object used in the quantum circuit qc
    theta (real): is the parameter parametrizing the trial state
    
    Return
    ---------
    qc: returns the input quantum circuit added with the gates creating the trial state
    
    """
    theta1 = params[0]
    theta2 = params[1] 
    theta3 = params[2]
    
    theta4 = params[3]
    theta5 = params[4] 
    theta6 = params[5]
    
    qc.h(qr[0])
    qc.h(qr[1])
    qc.h(qr[2])
    
    qc.cy(qr[0],qr[1])
    qc.ry(theta1, qr[0])
    qc.ry(theta2, qr[1])
    qc.ry(theta3, qr[2])
    
    qc.rz(theta4, qr[0])
    qc.rz(theta5, qr[1])
    qc.rx(theta6, qr[2])
    
    qc.sdg(qr[2])
    
    qc.cy(qr[2],qr[0])
    qc.sdg(qr[0])
#     qc.x(qr[0])
    
    return qc

def measurements(qc, qr, cr, op):
    """
    Implements the quantum measurements in different basis: XX, YY and ZZ.
    
    Arguments
    -----------
    qc: is a QuantumCircuit object from Qiskit
    qr: is a QuantumRegister object used in the quantum circuit qc
    cr: is a ClassicalRegister object used in the quantum circuit qc
    op (str): is a string with possible values: XX, YY and ZZ.
    
    
    Return
    ---------
    qc: returns the input quantum circuit added with the appropriate gates to measure in the selected basis.
    
    """
    
    if op == "XXI":
        # Change of basis, since X = HZH
        qc.h(qr[0])
        qc.h(qr[1])
        
        # CNOT used to measure ZZ operator
        qc.cx(qr[0],qr[1])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[1],cr[0])
        
    elif op == "XIX":
        # Change of basis, since X = HZH
        qc.h(qr[0])
        qc.h(qr[2])
        
        # CNOT used to measure ZZ operator
        qc.cx(qr[0],qr[2])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[2],cr[0])
        
    elif op == "IXX":
        # Change of basis, since X = HZH
        qc.h(qr[1])
        qc.h(qr[2])
        
        # CNOT used to measure ZZ operator
        qc.cx(qr[1],qr[2])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[2],cr[0])
        
    elif op == "YYI":
        # Change of basis, since Y = (HS†)Z(HS†)
        qc.sdg(qr[0])
        qc.sdg(qr[1])
        qc.h(qr[0])
        qc.h(qr[1])
        
        # CNOT used to measure ZZ operator
        qc.cx(qr[0],qr[1])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[1],cr[0])
        
    elif op == "YIY":
        # Change of basis, since Y = (HS†)Z(HS†)
        qc.sdg(qr[0])
        qc.sdg(qr[2])
        qc.h(qr[0])
        qc.h(qr[2])
        
        # CNOT used to measure ZZ operator
        qc.cx(qr[0],qr[2])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[2],cr[0])
    
    elif op == "IYY":
        # Change of basis, since Y = (HS†)Z(HS†)
        qc.sdg(qr[1])
        qc.sdg(qr[2])
        qc.h(qr[1])
        qc.h(qr[2])
        
        # CNOT used to measure ZZ operator
        qc.cx(qr[1],qr[2])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[2],cr[0])
        
    elif op == "ZZI":
        # CNOT used to measure ZZ operator
        qc.cx(qr[0],qr[1])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[1],cr[0])
     
    elif op == "ZIZ":
        # CNOT used to measure ZZ operator
        qc.cx(qr[0],qr[2])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[2],cr[0])
        
    elif op == "IZZ":
        # CNOT used to measure ZZ operator
        qc.cx(qr[1],qr[2])
        
        # Measurement of qubit 1 on classical register 0
        qc.measure(qr[2],cr[0])
    
    else:
        print(f"WARNING: Measurement on the {op} basis not supported")
        return 
        
    return qc

def hamiltonian(params):
    """
    Evaulates the Energy of the trial state using the mean values of the operators XX, YY and ZZ.
    
    Arguments
    -----------
    params (dict): is an dictionary containing the mean values form the measurements of the operators XX, YY, ZZ;
    
    Return
    ---------
    en (real): energy of the system
    
    """
    en = params['ZZI'] + params['XXI'] + params['YYI'] 
    +params['ZIZ'] + params['XIX'] + params['YIY'] 
    +params['IZZ'] + params['IXX'] + params['IYY'] 
    return en

def vqe_step(params):
    """
    Executes the VQE algorithm. 
    Creates and executes three quantum circuits (one for each of the observables XX, YY and ZZ), then evaluates the energy.
    
    Arguments
    -----------
    theta (real): is the parameter parametrizing the trial state
    
    Return
    --------
    energy (real): the energy of the system
    qc_list (dict): a dictionary containing the three quantum circuits for the observables XX, YY and ZZ
    
    """
#     print(params)
    # Number of executions for each quantum circuit
    shots=8192
    
    vqe_res = dict()
    qc_list = dict()
    
    for op in ["XXI", "YYI", "ZZI","XIX", "YIY", "ZIZ","IXX", "IYY", "IZZ"]:
        qr = qk.QuantumRegister(3, "qr")
        cr = qk.ClassicalRegister(1, "cr")
        qc = qk.QuantumCircuit(qr, cr)

        # Implementation of the ansatz
        qc = ansatz(qc, qr, params)

        # Just for plotting purposes
        qc.barrier()

        # Measurements in the appropriate basis (XX, YY, ZZ) are implemented
        qc = measurements(qc, qr, cr, op)
                
        # Get the measurements results
        counts = qk.execute(qc, sim_bknd, shots=shots).result().get_counts()

        # Check the results, and evaluate the mean value dividing by the number of shots
        if len(counts) == 1: 
            try:
                counts['0']
                mean_val = 1
            except:
                mean_val = -1
        else:
            # Evaluates the mean value of Z operator, as the difference in the number of 
            # 0s and 1s in the measurement outcomes
            mean_val = (counts['0']-counts['1'])/shots
            
        vqe_res[op] = mean_val
        qc_list[op] = qc
        
    energy = hamiltonian(vqe_res)
#     print("Mean values from measurement results:\n", vqe_res)
    
    print(params, energy)
#     print(energy)
    return energy
    

Let's try if it all works properly:

In [37]:
# Set the value of theta
theta = 0.2
theta2 = 0.4
theta3 = 0.5
params = [theta, theta2, theta3,np.pi,np.pi,np.pi]
# Run the VQE step to evaluate the energy (eigenvalue of the Hamiltonian) of the state with given theta
energy = vqe_step(params)


[0.2, 0.4, 0.5, 3.141592653589793, 3.141592653589793, 3.141592653589793] -1.0732421875


We see that for $\theta=0.2$, the energy $E(0.2)=\langle\mathcal{H}\rangle_{0.2} \sim 0.98$. Our aim is to find a value for the parameter that yields the lowest possible energy, and that is the desired lowest eigenvalue for the Task.  
The minimization procedure can be done by hand, or by setting up an optimizator. 

##### Using an optimizator

In [38]:
# minimize_scalar(vqe_step, args=(False), bounds = (0, pi), method = "bounded")
initial_guess = [np.pi, np.pi, np.pi,np.pi,np.pi,np.pi]
result = minimize(vqe_step, initial_guess , bounds =[(0., np.pi), (0., np.pi), (0., np.pi),(0., np.pi), (0., np.pi), (0., np.pi)], method="Powell")

[3.14159265 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] -0.0146484375
[1.19998161 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] -0.031494140625
[1.94161104 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.031005859375
[0.74162942 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.013916015625
[1.29501492 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] -0.01171875
[1.06005063 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.006103515625
[1.19622316 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.006591796875
[1.23628111 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.003662109375
[1.21384679 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] -0.023681640625
[1.20527764 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.029052734375
[1.20200452 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.015380859375
[1.19854601 3.14159265 3.14159265 3.14159265 3.14159265 3.14159265] 0.011962890625
[1.200

[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 2.85831542] -1.675537109375
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 2.9665177 ] -1.6904296875
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.04270315] -1.69921875
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.08047558] -1.710205078125
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.10382022] -1.718017578125
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.11824801] -1.69287109375
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.09490336] -1.714599609375
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.10933115] -1.721435546875
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.11273709] -1.68798828125
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.10722616] -1.7216796875
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.10801037] -1.7216796875
[1.20082397 1.559612   1.48149971 3.03037126 2.94294099 3.10761826] -1.735595703125
[1.2

[0.02483136 1.52250119 1.57195623 2.95158244 2.39996323 3.10761826] -2.697265625
[0.02483136 1.52250119 1.57195623 2.95158244 2.68324046 3.10761826] -2.9189453125
[0.02483136 1.52250119 1.57195623 2.95158244 3.01376321 3.10761826] -2.990966796875
[0.02483136 1.52250119 1.57195623 2.95158244 2.9669359  3.10761826] -2.99658203125
[0.02483136 1.52250119 1.57195623 2.95158244 2.93999815 3.10761826] -2.99462890625
[0.02483136 1.52250119 1.57195623 2.95158244 2.9673647  3.10761826] -2.99658203125
[0.02483136 1.52250119 1.57195623 2.95158244 2.9671503  3.10761826] -2.994140625
[0.02483136 1.52250119 1.57195623 2.95158244 2.98508735 3.10761826] -2.99609375
[0.02483136 1.52250119 1.57195623 2.95158244 2.97413415 3.10761826] -2.99609375
[0.02483136 1.52250119 1.57195623 2.95158244 2.9699504  3.10761826] -2.99462890625
[0.02483136 1.52250119 1.57195623 2.95158244 2.96835235 3.10761826] -2.995849609375
[0.02483136 1.52250119 1.57195623 2.95158244 2.96774194 3.10761826] -2.996337890625
[0.02483136 

[0.00784751 1.56337063 1.94161104 2.95044457 2.96780777 3.12914334] -2.866455078125
[0.00784751 1.56337063 2.39996323 2.95044457 2.96780777 3.12914334] -2.357177734375
[0.00784751 1.56337063 1.57186102 2.95044457 2.96780777 3.12914334] -2.99951171875
[0.00784751 1.56337063 1.57181134 2.95044457 2.96780777 3.12914334] -2.999755859375
[0.00784751 1.56337063 1.42978502 2.95044457 2.96780777 3.12914334] -2.98291015625
[0.00784751 1.56337063 1.50247224 2.95044457 2.96780777 3.12914334] -2.995361328125
[0.00784751 1.56337063 1.53758351 2.95044457 2.96780777 3.12914334] -2.997314453125
[0.00784751 1.56337063 1.55494261 2.95044457 2.96780777 3.12914334] -2.99951171875
[0.00784751 1.56337063 1.56340181 2.95044457 2.96780777 3.12914334] -2.999267578125
[0.00784751 1.56337063 1.56859919 2.95044457 2.96780777 3.12914334] -2.99951171875
[0.00784751 1.56337063 1.5705844  2.95044457 2.96780777 3.12914334] -2.999267578125
[0.00784751 1.56337063 1.57134269 2.95044457 2.96780777 3.12914334] -2.999511718

[0.01572529 1.19998161 1.57120539 2.97723473 2.98376799 3.12281586] -2.858642578125
[0.01572529 1.94161104 1.57120539 2.97723473 2.98376799 3.12281586] -2.855712890625
[0.01572529 0.74162942 1.57120539 2.97723473 2.98376799 3.12281586] -2.386962890625
[0.01572529 1.56850194 1.57120539 2.97723473 2.98376799 3.12281586] -2.999267578125
[0.01572529 1.56888486 1.57120539 2.97723473 2.98376799 3.12281586] -2.999755859375
[0.01572529 1.71125359 1.57120539 2.97723473 2.98376799 3.12281586] -2.981689453125
[0.01572529 1.63360898 1.57120539 2.97723473 2.98376799 3.12281586] -2.995849609375
[0.01572529 1.59977579 1.57120539 2.97723473 2.98376799 3.12281586] -2.9990234375
[0.01572529 1.58404488 1.57120539 2.97723473 2.98376799 3.12281586] -2.9990234375
[0.01572529 1.57618117 1.57120539 2.97723473 2.98376799 3.12281586] -2.999267578125
[0.01572529 1.57234155 1.57120539 2.97723473 2.98376799 3.12281586] -2.99951171875
[0.01572529 1.5702052  1.57120539 2.97723473 2.98376799 3.12281586] -2.9995117187

##### Conclusion
We see that the optimizator succeds in finding a solution. In particular, the optimal value seems to be $\theta \sim \pi$, and its energy is close to unity. 
We can then check this result directly feeding $\theta = \pi$ to the `vqe_step` function:

In [40]:
params=[0.01572529, 1.56938919, 1.56810936, 2.97185532, 2.98376799, 3.12281586] 
lowest = vqe_step(params)

[0.01572529, 1.56938919, 1.56810936, 2.97185532, 2.98376799, 3.12281586] -2.99951171875


**Thus, we found the solution to the Task, that is the lowest eigenvalue of $\mathcal{H}$ (formerly $U$), which amounts to $-2.999$.**

###### Final (classical) check
As a final step, we can check the result of our calculation with Scipy's eigensolver.

In [ ]:
# Definition of one qubit Pauli matrices
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

# Evaluation of two qubit Pauli matrices

XX = np.kron(X,X)
IXX = np.kron(I,XX)
XXI = np.kron(XX,I)
XI = np.kron(X,I)
XIX = np.kron(XI,X)

YY = np.kron(Y,Y)
IYY = np.kron(I,YY)
YYI = np.kron(YY,I)
YI = np.kron(Y,I)
YIY = np.kron(YI,Y)

ZZ = np.kron(Z,Z)
IZZ = np.kron(I,ZZ)
ZZI = np.kron(ZZ,I)
ZI = np.kron(Z,I)
ZIZ = np.kron(ZI,Z)

# Calculation of the Hamiltonian
H = (ZZI) + (XXI) +(YYI) +(ZIZ) + (XIX) +(YIY) +(IZZ) + (IXX) +(IYY)

print("Desired Hamiltionian H = \n", H)

Which is the same as $U$. Now we proceed calculating the eigenvalues:

In [ ]:
import scipy

# Calculate eigenvalues and eigenvectors of H
eigenvalues, eigenvectors = scipy.linalg.eig(H)
print("Eigenvalues:", eigenvalues)
# print("EVecs:", eigenvectors)
print("Min Eigenvalues:", np.real(min(eigenvalues)))

We see that $-3.0$ is actually the lowest eigenvalue, as we found with VQE.